In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time
import operator
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [2]:
#Stanting timming process
start = time.time()

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
# subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]
subfolder_with_path = ["" for i in range(4)]
subfolder_with_path[0] = main_directory + '/'+'01Activity_files'
subfolder_with_path[1] = main_directory + '/'+'02Volume_files'
subfolder_with_path[2] = main_directory + '/'+'03Volume_in_tissue_files'
subfolder_with_path[3] = main_directory + '/'+'zzz_input_and_reference_files'

In [5]:
###############################################################################################
# Importing information from "samples overview", "intensisties template" and "brain gubra map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])


# Reading "sample_overview.csv" file
samples_overview = pd.read_csv("sample_overview.csv", 
                               dtype={"SAMPLE": int,
                                      "hemisphere": "string","marker": "string",
                                      "TX_GROUP": "string"})

# Reading "gubra_intensities_template.csv" file
gubra_intensities_template = pd.read_csv("gubra_intensities_template.csv",
                                   dtype={"IDPath": "string", 
                                          "LabelID": int, 
                                          "raw": "string", 
                                          "LabelAbrv": "string"})
# Reading "brain_allen_map.csv" file
brain_gubra_map = pd.read_csv("brain_gubra_map_fixed.csv", 
                               dtype={"IDPath": "string", "LabelAbrv": "string",
                                      "raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})


In [6]:
#Saving drugs and markers' names from the "sample_overview" file

drugs = samples_overview['TX_GROUP'].unique()
markers = samples_overview['marker'].unique()

d = 0
m = 0
counters_names = []
while (m < len(markers)):
    while (d < len(drugs)):
        counters_names.append(drugs[d] + '-' + markers[m])
        d += 1
    m += 1
    d = 0

In [7]:
# Changing directory to the "01Activity_files" folder
os.chdir(subfolder_with_path[0])

#Counting number of files on "01Activity_files" folder
list = os.listdir(subfolder_with_path[0]) 
number_files = len(list)
number_activity_files = number_files

In [8]:
#Using and timing  a loop to process all files and populate the main data-frame

activity_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

#Zeroing the counters 
counters = np.zeros(len(counters_names))

#Creating counters array to contain sample's names as elements
counters_samples = [[] for j in range(len(counters_names))]


    ##############################################
    # Innitiates loop to process each file
    ##############################################


while i < number_files:

    # Working with each file at a time
    file = list[i]
    
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]        

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, 
                                                   ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
#     total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)
        
    # Adding activity columns to intensitites template data-frame
    output = gubra_intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    if hemisphere[0] == 'right':
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE',
                                             'hemisphere','marker',
                                             'TX_GROUP']],on='SAMPLE',how='inner')
    
    
    #Adding a column with the sample's nickname

    counters_index = counters_names.index(output['TX_GROUP'].iloc[0] + '-' + output['marker'].iloc[0])
    if output['SAMPLE'].iloc[0] not in counters_samples[counters_index]:
        counters[counters_index] += 1
        counters_samples[counters_index].append(output['SAMPLE'].iloc[0])
        nickname = output['TX_GROUP'].iloc[0] + '-' + str(int(counters[counters_index]))

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname
    
    # In case the sample has both hemispheres
    if hemisphere[0] == 'both':
        output['nickname'] = output['nickname'] + '-' + output["LabelAbrv"].str[0]

    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('L','',1)
    output["LabelAbrv"] = output["LabelAbrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    activity_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
activity_data = pd.concat(activity_data)

In [9]:
#Replacing empty cells with zeros
activity_data['ACTIVITY'].fillna(0, inplace=True)

In [10]:
# Changing directory to the main path
os.chdir(main_directory)

In [11]:
########Creating output for Dan's Excel analysis


#Chaging SAMPLE elements to integer and then to string when adding the letter "S"
activity_data['SAMPLE_STR'] = activity_data['SAMPLE'].astype(int)
activity_data['SAMPLE_STR'] = activity_data['SAMPLE_STR'].astype(str)
activity_data['SAMPLE_STR'] = 'S' + activity_data['SAMPLE_STR']

#Pivoting dataframe to have sample as columns
pivoted_activity_Dan = pd.pivot_table(activity_data, values='ACTIVITY', 
                               index=['raw','hemisphere',
                                      'marker','SAMPLE'],
                                      columns=['SAMPLE_STR','nickname'], aggfunc=np.sum)

#Restarting index to activate'hemisphere','marker' as columns
pivoted_activity_Dan = pivoted_activity_Dan.reset_index()

#Adding GUBRA map columns
gubra_and_activity_Dan = pd.merge(brain_gubra_map,pivoted_activity_Dan,on='raw',how='right')

#Editing output
gubra_and_activity_Dan.rename(columns={('SAMPLE', ''): 'SAMPLE',
                                       ('hemisphere', ''): 'hemisphere',
                                       ('marker', ''): 'marker',
                                       'raw': 'LabelName'}, inplace=True)
gubra_and_activity_Dan = gubra_and_activity_Dan.drop(columns=['allen_1', 'allen_2',
                                     'allen_3', 'allen_4',
                                     'allen_5', 'allen_6',
                                     'allen_7', 'fine',
                                     'medium', 'coarse','all',
                                    'SAMPLE', 'hemisphere','marker'])
gubra_and_activity_Dan = gubra_and_activity_Dan.groupby('IDPath').apply(lambda x: x\
                                                                        .fillna(method='ffill')\
                                                                        .fillna(method='bfill')\
                                                                        .drop_duplicates()\
                                                                        .reset_index(drop=True))

C:\Users\greci\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\greci\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [12]:
#editing columns names from ('sample43', 'psilocybin') to S43_psilocybin
columns_name_array = gubra_and_activity_Dan.columns.values
#dividing into two arrays, one with str and one with tuple
#"cna" = column_name+array
number_of_cols = columns_name_array.size
cna_1 = columns_name_array[0:3]
cna_2 = columns_name_array[-(number_of_cols - 3):]
str = ''
for item in cna_2:
    str = str +'_'.join(item) + ' ' 
str = str.split(' ')[:-1]
new_columns_name = np.concatenate([cna_1, str]) 
new_columns_name
#Asingning new column names to the dataframe
gubra_and_activity_Dan = gubra_and_activity_Dan.set_axis(new_columns_name, axis=1, inplace=False)

In [13]:
#Ordering columns so all saline samples are fisrt
cols = gubra_and_activity_Dan.columns.tolist()
cols_first_3 = cna_1
cols_samples = cols[-(number_of_cols - 3):]

def w_len(e):
    len_e = len(e)
    if operator.contains(e, "aline"):
        len_e = 1
    return len_e

cols_samples.sort(key=w_len)
new_cols = np.concatenate((cols_first_3, cols_samples), axis=None)

gubra_and_activity_Dan = gubra_and_activity_Dan[new_cols]

In [14]:
gubra_and_activity_Dan.to_csv('GUBRA_output_cc_Dan_Excel.csv', index=False)

In [15]:
#Ending timming process
end = time.time()

# Printing how many files were processed and how much time the process took
files_processed = number_activity_files = number_files
print(files_processed,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

36  Files processed 
 
 Execution time: 120.93 seconds
